# This is the python template for Assignment 04.  
- You must use this template.  
- You must not change any signatures of the methods, only edit the sections indicated with "Write your code here."  
- The return of every function has to be in the right format, otherwise this is a desk reject.  
- Plagiarism leads to failing the assignment!  
- We will terminate the script after 10 min, try to use efficient algorithms.

In [3]:
import pandas as pd

In [4]:
def get_name():
    return "Irem Begüm Gündüz"
def get_matriculationnumber():
    return 7026821

## Useful information:

The structure of a CART is a dict. Use the same names as shown in the example, using other names makes your format invalid and leads to a desk reject.

In [ ]:
cart = { "name":"X", "mean":456, "split_by_feature": "aes", "error_of_split": 0.0,
        "successor_left": { "name":"XL", "mean":1234, "split_by_feature": None, "error_of_split":None,
                           "successor_left":None,
                           "successor_right":None
                          },
        "successor_right":{ "name":"XR", "mean":258, "split_by_feature": None,"error_of_split":None,
                           "successor_left":None,
                           "successor_right":None
                          }
       }

The names of the features must be used as defined in this list, using other names makes your format invalid and leads to a desk reject.

In [ ]:
features = ["secompress", "encryption", "aes", "blowfish", "algorithm", "rar", "zip", "signature",
            "timestamp", "segmentation", "onehundredmb", "onegb"]

# Task 1: Create a CART

In [18]:
def compute_performance(dataframe,dataframe2=pd.DataFrame(),metric= "performance"):
  #compute the performance of given dataframe(S)
    if  dataframe2.empty:
         feature = extract_features(dataframe)
         results_table = compute_performance_helper(dataframe,feature, metric)
         return results_table
    else:
       features,features2 = extract_features(dataframe,dataframe2)
       results_table = compute_performance_helper(dataframe,features, metric)
       results_table2 = compute_performance_helper(dataframe2,features2, metric)
       return results_table, results_table2

def compute_performance_helper(dataframe,features, metric= "performance"):
    # Initialize a results dataframe
    results_table = results_helper_df()
    
    for feature in features:
        # Split the dataframe into two based on the feature
        left_data = dataframe[dataframe[feature] == 1]
        right_data = dataframe[dataframe[feature] == 0]

        # Count the number of observations in each split
        left_count = len(left_data)
        right_count = len(right_data)

        # Sum the values of the metric in each split
        left_sum = left_data[metric].sum()
        right_sum = right_data[metric].sum()

        # Calculate the mean of the metric in each split
        if left_count != 0: 
          left_mean = round(left_sum / left_count,2) 
        else: 
          left_mean = 0

        if right_count != 0: 
          right_mean = round(right_sum / right_count,2)
        else: 
          right_mean = 0

        # Calculate the squared error of the metric in each split
        left_sq_err = ((left_data[metric] - left_mean) ** 2).sum()
        right_sq_err = ((right_data[metric] - right_mean) ** 2).sum()
        left_sq_err, right_sq_err = round(left_sq_err, 2), round(right_sq_err, 2)

        # Calculate the total squared error and mean of the metric
        total_sq_err = left_sq_err + right_sq_err
        total_mean = round((left_sum + right_sum) / (left_count + right_count), 2)

        # Create a result list and append it to the results DataFrame
        result = [feature, left_count, left_mean, left_sq_err, right_count, right_mean, right_sq_err, total_mean, total_sq_err]
        results_table.loc[len(results_table)] = result
    return results_table 

def results_helper_df():
    #create an empty dataframe
    results_df = pd.DataFrame(columns=['feature', 'left_count', 'left_mean', 
                                       'left_sq_err', 'right_count', 'right_mean',
                                       'right_sq_err', 'total_mean', 'total_sq_err'])
    return results_df

def extract_features(df,df2=pd.DataFrame()):
    #extract features from columns by disgarding id and performance
    features = [col for col in df.columns if col != 'performance' and col != 'Id']
    if  df2.size == 0: 
      return features
    else :
      features2 = [col for col in df2.columns if col != 'performance' and col != 'Id']
      return features,features2

def get_split_var(df, tot_error_col='total_sq_err', feature_col='feature'):
  # check if any value in the tot_error_col is None
  if df[tot_error_col].isna().any() or df[tot_error_col].min() == None:
    return None
  # check if minimum value in the tot_error_col is None
  min_error = df[tot_error_col].min()

  # Count the number of times the minimum value appears in the column
  counts_min = df[tot_error_col].value_counts()[min_error]

  # if there is only one feature with the minimum error extract the split var
  if counts_min == 1:
    split_var = df.loc[df[tot_error_col] == min_error, feature_col].values[0]
  else:
    df_min = df.query(f'{tot_error_col} == @min_error')
    #if all features have the same min error return none
    if df_min[feature_col].nunique() == df[feature_col].nunique():
       return None
     #else sort the features alphabetically, return the first one
    split_var = df_min[feature_col].sort_values().values[0]
  return split_var


def build_tree(df, results=pd.DataFrame(), feature_col='feature', totError='total_sq_err', name='X'):
  if results.empty:
    #compute the initial performance 
    results = compute_performance(df, metric = 'performance')
    
  #start an empty tree and name the first value splitting as X
  tree = {}
  tree['name'] = name
  tree['mean'] = df['performance'].mean()
  splitting_var = get_split_var(results)
    
  # If no more splits are possible
  if splitting_var is None:
    tree['split_by_feature'] = None
    tree['error_of_split'] = None
    tree['successor_left'] = None
    tree['successor_right'] = None

  else:
    tree['split_by_feature'] = splitting_var
    tree['error_of_split'] = results[results[feature_col] == splitting_var][totError].values[0]
    left_df = df.query(f'{splitting_var} == 1')
    left_df = left_df.drop(splitting_var, axis = 1)
    right_df = df.query(f'{splitting_var} == 0')
    right_df = right_df.drop(splitting_var, axis = 1)
    
    # Compute the performance of the left and right splits
    results_left,results_right = compute_performance(left_df,right_df)
    
    # Recursively call the function to build the tree
    tree['successor_left'] = build_tree(left_df, results_left, name=name+'L')
    tree['successor_right'] = build_tree(right_df, results_right, name=name+'R')
        
  return tree


In [19]:
def get_cart(sample_set_csv):
    # The sample_set_csv is a file path to a csv data, this can be imported into a dataframe
    df = pd.read_csv(sample_set_csv)
    
    #build the cart tree
    cart = build_tree(df)

    return cart


# Task 2a: Highest influencing feature

In [ ]:
# Write your helper functions here, if needed

In [ ]:
def get_highest_influence_feature(cart):
    # TODO: Write your code here. And change the return.
    return "secompress"

# Task 2b: Calculate the error rate

In [ ]:
# Write your helper functions here, if needed

In [ ]:
def get_error_rate(cart, sample_set_csv):
    # The sample_set_csv is a file path to a csv data, this can be imported into a dataframe
    df = pd.read_csv(sample_set_csv)
    # TODO: Write your code here. And change the return.
    return 42

# Task 2c: Generate optimal configuration

In [ ]:
# Write your helper functions here, if needed

In [ ]:
def get_optimal_configuration(cart, partial_configuration):
    # TODO: Write your code here. And change the return.
    return {"rar", "timestamp"}

# Tests:  
In the following cells, we provide you some test cases (but not all possible test cases!)

In [21]:
# Task 1

test_cart = {'name': 'X', 'mean': 763.2, 'split_by_feature': 'segmentation', 'error_of_split': 6.0, 
             'successor_left': 
                 {'name': 'XL', 'mean': 772.0, 'split_by_feature': 'onegb', 'error_of_split': 0.0, 
                  'successor_left': 
                      {'name': 'XLL', 'mean': 770.0, 'split_by_feature': None, 'error_of_split': None, 
                       'successor_left': None, 
                       'successor_right': None
                      }, 
                  'successor_right': 
                      {'name': 'XLR', 'mean': 773.0, 'split_by_feature': None, 'error_of_split': None, 
                       'successor_left': None, 
                       'successor_right': None
                      }
                 }, 
             'successor_right': 
                 {'name': 'XR', 'mean': 750.0, 'split_by_feature': None, 'error_of_split': None, 
                  'successor_left': None, 
                  'successor_right': None}
            }


if get_cart("Performance_01.csv") == test_cart:
    print("passed")
else:
    print("failed")

passed


In [ ]:
# Task 2b
if get_error_rate(test_cart, "Performance_02b.csv") == 5:
    print("passed")
else:
    print("failed")

In [ ]:
# Task 2c
test_cart_v2 = {'name': 'X', 'mean': 763.2, 'split_by_feature': 'zip', 'error_of_split': 0.0, 
                 'successor_left': {'name': 'XL', 'mean': 772.0, 'split_by_feature': None, 'error_of_split': None, 
                                    'successor_left': None, 
                                    'successor_right': None}, 
                 'successor_right': {'name': 'XR', 'mean': 750.0, 'split_by_feature': None, 'error_of_split': None, 
                                     'successor_left': None, 
                                     'successor_right': None}
                }

optimal_config = get_optimal_configuration(test_cart_v2, {"secompress", "encryption", "aes", "algorithm", "signature",
                                                        "timestamp", "segmentation", "onehundredmb"})
reference = {'aes', 'algorithm', 'encryption', 'onehundredmb', 'rar', 'secompress', 'segmentation', 'signature',
            'timestamp'}

if optimal_config == reference:
    print("passed")
else:
    print("failed")